STEP 1: DATA LOADING AND PREPROCESSING
**bold text**

In [2]:
import json


In [ ]:
#filters out invalid json objects and saves the remaining in output file. the try block might
#raise an exception so to handle that the except block executes

json_file = '/content/Cell_Phones_and_Accessories_5.json'
output_json_list = []

try:
    with open(json_file, 'r') as file:
        for line in file:
            try:
                python_data = json.loads(line)
                output_json_list.append(python_data)
            except json.JSONDecodeError:
                print("Ignoring line}")

    for data in output_json_list:
        print(data)
except FileNotFoundError:
    print("File not found.")
except Exception as e:
    print("An unexpected error occurred")


In [ ]:
#structure of the dataset where the key from the key-value pair is printed
if output_json_list:
        first_record = output_json_list[0]
        print("Keys in the first record:", list(first_record.keys()))
print("\n")


#total number of records in the dataset by printing length of it
num_records = len(output_json_list)
print("Number of records:", num_records)

#printing the data types where items function is used to iterate over key-value pair
if output_json_list:
        first_record = output_json_list[0]
        second_record = output_json_list[1]
        third_record = output_json_list[2]
        fourth_record = output_json_list[3]
        fifth_record = output_json_list[4]


        print("\nStructure of the first record:")
        for key, value in first_record.items():
            print(key, type(value))

        print("\nStructure of the second record:")
        for key, value in second_record.items():
            print(key, type(value))

        print("\nStructure of the thrid record:")
        for key, value in third_record.items():
            print(key, type(value))

        print("\nStructure of the fourth record:")
        for key, value in fourth_record.items():
            print(key, type(value))

        print("\nStructure of the fifth record:")
        for key, value in fifth_record.items():
            print(key, type(value))


In [ ]:
#filtering out columns to keep necessary information to avoid unnecessary computation error
columns = ['reviewText', 'overall', 'summary']
filtered_dataset=[]
for record in output_json_list:
    filtered_data = {}
    for key in columns:
        filtered_data[key] = record[key]
    filtered_dataset.append(filtered_data)

print("The Filtered Dataset is as shown:")
for i in range(min(50, len(filtered_dataset))):

    print(filtered_dataset[i])


In [ ]:
#applying text preprocessing techniques to remove punctuations, stop word and lower the case
def is_punctuation(char):
    return char in {'!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~'}

def preprocess_text(text):
    text = text.lower()
    filtered_text = ''
    for char in text:
        if not is_punctuation(char):
            filtered_text += char
    text = filtered_text

    stopwords_list = [
    'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves',
    'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their',
    'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was',
    'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and',
    'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between',
    'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off',
    'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both',
    'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too',
    'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'now']

    words = text.split()
    filtered_words = []
    for word in words:
     if word not in stopwords_list:
        filtered_words.append(word)

    words = filtered_words
    text = ''
    for word in words:
       text += word + ' '
    print(text)

    return text


for record in output_json_list:
    filtered_data = {}
    for key in columns:
        if key == 'reviewText':
          record[key] = preprocess_text(record[key])
        filtered_data[key] = record[key]

        if key == 'summary':
          record[key] = preprocess_text(record[key])
        filtered_data[key] = record[key]

    filtered_dataset.append(filtered_data)

print("Filtered Dataset:")
for i in range(min(50, len(filtered_dataset))):
    print(filtered_dataset[i])





STEP 2: THEMATIC ANALYSIS


In [ ]:
def identify_sentiment(text):
    positive_sentiment = ['good', 'great', 'excellent', 'positive', 'happy','nice','amazing']
    negative_sentiment = ['bad', 'poor', 'negative', 'disappointing', 'unhappy','low','fake']

    positive_count=0
    for word in preprocess_text(text).split():
     if word in positive_sentiment:
      positive_count += 1
     else:
      positive_count=0

    negative_count=0
    for word in preprocess_text(text).split():
     if word in negative_sentiment:
      negative_count +=1
     else:
      negative_count=0

    if positive_count > negative_count:
     return 'positive'
    else:
     return 'negative'


def identify_keyphrases(text, num_phrases=5):
    words = text.split()
    key_phrases = {}
    for word in words:
        key_phrases[word] = key_phrases.get(word, 0) + 1
    sorted_key_phrases = sorted(key_phrases.items(), key=lambda x: x[1], reverse=True)[:num_phrases]
    return sorted_key_phrases

columns = ['reviewText', 'summary', 'overall']
filtered_dataset = []
for record in output_json_list:
    filtered_data = {}
    for key in columns:
        #record[key] = preprocess_text(record[key])
        if key == 'reviewText':
            key_phrases = identify_keyphrases(record[key])
            filtered_data['key_phrases'] = key_phrases
        elif key == 'summary':
            key_phrases_summary = identify_keyphrases(record[key])
            filtered_data['key_phrases_summary'] = key_phrases_summary
        filtered_data[key] = record[key]

    sentiment = identify_sentiment(record['reviewText'])
    filtered_data['sentiment'] = sentiment

    filtered_dataset.append(filtered_data)


print("Filtered Dataset:")
for i in range(min(50, len(filtered_dataset))):
    print(filtered_dataset[i])

In [27]:
def identify_sentiment(overall_rating):
#assuming ratings greater than 5 are positive, else negative
    return 'positive' if overall_rating > 5 else 'negative'

def identify_keyphrases(text, positive_phrases, negative_phrases, sentiment):
    words = text.split()
    for word in words:
        if sentiment == 'positive':
            positive_phrases[word] = positive_phrases.get(word, 0) + 1
        else:
            negative_phrases[word] = negative_phrases.get(word, 0) + 1

columns = ['reviewText', 'summary', 'overall']
positive_keywords = {}
negative_keywords = {}

for record in output_json_list:
    sentiment = identify_sentiment(record['overall'])
    for key in columns:
        if key == 'reviewText':
            identify_keyphrases(record[key], positive_keywords, negative_keywords, sentiment)
        elif key == 'summary':
            identify_keyphrases(record[key], positive_keywords, negative_keywords, sentiment)

#xxtract a few words from both positive and negative keywords
positive_keywords = [word for word, count in positive_keywords.items() if count > 1]
negative_keywords = [word for word, count in negative_keywords.items() if count > 1]

#calculate the CDF for positive and negative keywords
positive_cdf = {word: sum(positive_keywords.count(w) for w in positive_keywords) for word in positive_keywords}
negative_cdf = {word: sum(negative_keywords.count(w) for w in negative_keywords) for word in negative_keywords}

print("Mixed Keywords:", positive_keywords + negative_keywords)
print("\nPositive CDF:", positive_cdf)
print("\nNegative CDF:", negative_cdf)


KeyboardInterrupt: 

SEMANTIC ANALYSIS


In [ ]:
def rule_based_sentiment_analysis(text, positive_weights, negative_weights):
    words = text.split()

    sentiment_score = 0
    for word in words:
        sentiment_score += positive_weights.get(word, 0) - negative_weights.get(word, 0)

    if sentiment_score > 0:
        sentiment = 'positive'
    elif sentiment_score < 0:
        sentiment = 'negative'
    else:
        sentiment = 'neutral'

    return sentiment, sentiment_score

#weights depending upon positive and negative words
positive_weights = {'good': 2, 'great': 3, 'excellent': 3, 'positive': 2, 'happy': 2, 'nice': 2, 'amazing': 4}
negative_weights = {'bad': -2, 'poor': -3, 'negative': -2, 'disappointing': -3, 'unhappy': -2, 'low': -2, 'fake': -3}

columns = ['reviewText', 'summary', 'overall']
filtered_dataset = []

for record in output_json_list:
    filtered_data = {}
    for key in columns:
        filtered_data[key] = record[key]

#Identify sentiment based on 'reviewText' column using rule-based analysis
    sentiment, sentiment_score = rule_based_sentiment_analysis(record['reviewText'], positive_weights, negative_weights)
    filtered_data['sentiment'] = sentiment
    filtered_data['sentiment_score'] = sentiment_score

    filtered_dataset.append(filtered_data)

for i in range(min(100, len(filtered_dataset))):
    print("Review Text:", filtered_dataset[i]['reviewText'])
    print("Overall:", filtered_dataset[i]['overall'])
    print("Summary:", filtered_dataset[i]['summary'])
    print("Sentiment:", filtered_dataset[i]['sentiment'])
    print("Sentiment Score:", filtered_dataset[i]['sentiment_score'])
    print("\n---\n")


In [25]:
#the text is saved to an other output file
output_file_path = 'sentiment_results.txt'
with open(output_file_path, 'w') as output_file:
    for data in filtered_dataset:
        output_file.write(f"Review Text: {data['reviewText']}\n")
        output_file.write(f"Sentiment: {data['sentiment']}\n")
        output_file.write("\n---\n")

print(f"Results saved to {output_file_path}")


Results saved to sentiment_results.txt
